# LOGISTICS PROBLEM IMPLEMENTATION USING METAHEURASTICS
TWO MAIN COMPONENTS AND THEIR ATTRIBUTES: 
1. ITEMS 
    1. item
    2. size | x_size, y_size
    3. delivery_loc
    4. delivery_time (deadline: 1st or 2nd round)
    5. delivery_dispatch (which round it is actually dispatched)
    6. delivery_number (on that route)
    7. bin

2. VEHICLES
    1. bin
    2. size | x_size, y_size
    3. available_space | av_x_space, av_y_space
    4. delivery_time_dispatch (in which round will the vehicle be dispatched)
    5. route_dist
    6. bin_penalty
    7. route_cost

PROBLEM STATEMENT:
The items have to be prioritiesed by the delivery time. The items are allocated to bins either in a greedy or metaheurastic fashion. For the metaheurastic case, the combination of items placed in a bin are checked to ensure that the bins are not overfilled (constraint). If the bins are filled the items are removed and are added to a new bin. If there is surplus space, then the lower priority items are too considered to be filled in the bins. 

Once all items have been allocated bins, we group the items which have been allocated to the same bin. These items' delivery loaction is too considered, and a route is generated either using a greedy or metaheurastic approach. The route order that is generated is mapped to the delivery number of the item. 

IMPLEMENTATION STEPS:
1. Priorities the items which have to be delivered in the first round
2. Check space availability:
a. Insufficient space, then we have have to allocate to a dummy bin. All these items then are reconsidered using the bin allocation technique.
b. Surplus space, then fill lower priority items in opened bins.
3. Group items by bin
4. Generate a route for the items in the bin according to their delivery_loc
5. Set the order of where the state occurs in the route to the item

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math 
from math import floor
from random import randint
import csv as csv
#to shuffle dataframe
from sklearn.utils import shuffle 
from IPython.display import display, HTML
import scipy 
from scipy.misc import comb # comb(n,k, exact=True)
from math import inf
from math import exp, expm1
import decimal
import random
# from scipy import special
# from scipy.special import comb
CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

### Read-in and check data

In [2]:
def read_data(fileName):
    df = pd.read_csv(fileName)
    return df
    
def check_packaging(df):
    rows, cols = df.shape #size of the data set
    return (rows, cols)

def data_check(df, n=3):#n number of items to check 
    df_top_n = df.head(n)
    return (df_top_n)

def check_ns(df):
    ns = df.describe()
    return ns

In [3]:
###FILE NAMES
#ITEMS: 
items = 'items'
bins = 'bins'
items_2D = 'items2D'
bins_2D = 'bins2D'
city_dist = 'city'
dist_mat = 'distance_matrix'

#### 1D Item data

In [4]:
df_items = read_data('%s.csv'%items)
print("rows(%s) x cols(%s) "%check_packaging(df_items))
print()
print("%s"%data_check(df_items))
print()
print(check_ns(df_items))
print()
df_items.set_index('item')

rows(8) x cols(7) 

   item  size  bin  delivery_loc  delivery_time  delivery_dispatch  \
0     0     4  NaN             4              2                NaN   
1     1     3  NaN             2              1                NaN   
2     2     6  NaN             3              2                NaN   

   delivery_number  
0              NaN  
1              NaN  
2              NaN  

          item      size  bin  delivery_loc  delivery_time  delivery_dispatch  \
count  8.00000  8.000000  0.0      8.000000       8.000000                0.0   
mean   3.50000  3.375000  NaN      3.000000       1.375000                NaN   
std    2.44949  1.846812  NaN      1.603567       0.517549                NaN   
min    0.00000  1.000000  NaN      1.000000       1.000000                NaN   
25%    1.75000  2.000000  NaN      1.750000       1.000000                NaN   
50%    3.50000  3.000000  NaN      3.000000       1.000000                NaN   
75%    5.25000  4.500000  NaN      4.250000    

,size,bin,delivery_loc,delivery_time,delivery_dispatch,delivery_number
item,,,,,,
0,4,NaN,4,2,NaN,NaN
1,3,NaN,2,1,NaN,NaN
2,6,NaN,3,2,NaN,NaN
3,3,NaN,5,1,NaN,NaN
4,2,NaN,1,1,NaN,NaN
5,1,NaN,1,2,NaN,NaN
6,2,NaN,5,1,NaN,NaN
7,6,NaN,3,1,NaN,NaN


#### 2D Item data

In [5]:
df_items2D = read_data('%s.csv'%items_2D)
print("rows(%s) x cols(%s) "%check_packaging(df_items))
print()
print("%s"%data_check(df_items))
print()
print(check_ns(df_items))
print()
df_items2D.set_index('item')

rows(8) x cols(7) 

   item  size  bin  delivery_loc  delivery_time  delivery_dispatch  \
0     0     4  NaN             4              2                NaN   
1     1     3  NaN             2              1                NaN   
2     2     6  NaN             3              2                NaN   

   delivery_number  
0              NaN  
1              NaN  
2              NaN  

          item      size  bin  delivery_loc  delivery_time  delivery_dispatch  \
count  8.00000  8.000000  0.0      8.000000       8.000000                0.0   
mean   3.50000  3.375000  NaN      3.000000       1.375000                NaN   
std    2.44949  1.846812  NaN      1.603567       0.517549                NaN   
min    0.00000  1.000000  NaN      1.000000       1.000000                NaN   
25%    1.75000  2.000000  NaN      1.750000       1.000000                NaN   
50%    3.50000  3.000000  NaN      3.000000       1.000000                NaN   
75%    5.25000  4.500000  NaN      4.250000    

,x_size,y_size,bin,delivery_loc,delivery_time,delivery_dispatch,delivery_number
item,,,,,,,
0,4,2,NaN,2,1,NaN,NaN
1,3,1,NaN,5,2,NaN,NaN
2,6,8,NaN,3,2,NaN,NaN
3,3,2,NaN,4,1,NaN,NaN
4,2,1,NaN,3,2,NaN,NaN
5,1,1,NaN,1,1,NaN,NaN
6,2,2,NaN,4,2,NaN,NaN


#### Vehicle data

In [6]:
df_vehicles = read_data('%s.csv'%bins)
print("rows(%s) x cols(%s) "%check_packaging(df_vehicles))
print()
print("%s"%data_check(df_vehicles))
print()
print(check_ns(df_vehicles))
print()
df_vehicles.set_index('bin')

rows(3) x cols(7) 

   bin  size  available_space  delivery_time_dispatch  route_dist  \
0    0     9                9                     NaN         NaN   
1    1     3                3                     NaN         NaN   
2    2     4                4                     NaN         NaN   

   bin_penalty  route_cost  
0          1.0         NaN  
1          0.8         NaN  
2          2.0         NaN  

       bin      size  available_space  delivery_time_dispatch  route_dist  \
count  3.0  3.000000         3.000000                     0.0         0.0   
mean   1.0  5.333333         5.333333                     NaN         NaN   
std    1.0  3.214550         3.214550                     NaN         NaN   
min    0.0  3.000000         3.000000                     NaN         NaN   
25%    0.5  3.500000         3.500000                     NaN         NaN   
50%    1.0  4.000000         4.000000                     NaN         NaN   
75%    1.5  6.500000         6.500000          

,size,available_space,delivery_time_dispatch,route_dist,bin_penalty,route_cost
bin,,,,,,
0,9,9,NaN,NaN,1.0,NaN
1,3,3,NaN,NaN,0.8,NaN
2,4,4,NaN,NaN,2.0,NaN


In [7]:
df_vehicles2D = read_data('%s.csv'%bins_2D)
print("rows(%s) x cols(%s) "%check_packaging(df_vehicles2D))
print()
print("%s"%data_check(df_vehicles2D))
print()
print(check_ns(df_vehicles2D))
print()
df_vehicles2D.set_index('bin')

rows(3) x cols(9) 

   bin  x_size  y_size  av_x_space  av_y_space  delivery_time_dispatch  \
0    0       9       6           9           6                     NaN   
1    1       5       3           5           3                     NaN   
2    2       7       5           7           5                     NaN   

   route_dist  bin_penalty  route_cost  
0         NaN          2.0         NaN  
1         NaN          1.0         NaN  
2         NaN          0.6         NaN  

       bin  x_size    y_size  av_x_space  av_y_space  delivery_time_dispatch  \
count  3.0     3.0  3.000000         3.0    3.000000                     0.0   
mean   1.0     7.0  4.666667         7.0    4.666667                     NaN   
std    1.0     2.0  1.527525         2.0    1.527525                     NaN   
min    0.0     5.0  3.000000         5.0    3.000000                     NaN   
25%    0.5     6.0  4.000000         6.0    4.000000                     NaN   
50%    1.0     7.0  5.000000         7

,x_size,y_size,av_x_space,av_y_space,delivery_time_dispatch,route_dist,bin_penalty,route_cost
bin,,,,,,,,
0,9,6,9,6,NaN,NaN,2.0,NaN
1,5,3,5,3,NaN,NaN,1.0,NaN
2,7,5,7,5,NaN,NaN,0.6,NaN


#### City data 
(Not all states are connected)

In [8]:
df_cityDist = read_data('%s.csv'%city_dist)
print("rows(%s) x cols(%s) "%check_packaging(df_cityDist))
print()
print("%s"%data_check(df_cityDist))
print()
print(check_ns(df_cityDist))
print()
num_cities = df_cityDist.shape[0]
# df_cityDist.set_index('city')
print('Number of cities including depot: ', num_cities)

rows(6) x cols(6) 

     0    1    2    3    4   5
0  NaN  NaN  1.0  3.0  NaN NaN
1  NaN  NaN  2.0  3.0  1.0 NaN
2  1.0  2.0  NaN  4.0  NaN NaN

              0    1         2         3    4    5
count  2.000000  3.0  4.000000  4.000000  1.0  2.0
mean   2.000000  2.0  2.250000  3.000000  1.0  2.0
std    1.414214  1.0  1.258306  0.816497  NaN  0.0
min    1.000000  1.0  1.000000  2.000000  1.0  2.0
25%    1.500000  1.5  1.750000  2.750000  1.0  2.0
50%    2.000000  2.0  2.000000  3.000000  1.0  2.0
75%    2.500000  2.5  2.500000  3.250000  1.0  2.0
max    3.000000  3.0  4.000000  4.000000  1.0  2.0

Number of cities including depot:  6


#### Symmetric distance matrx

In [9]:
df_distMat = read_data('%s.csv'%dist_mat)
print("rows(%s) x cols(%s) "%check_packaging(df_distMat))
print()
print("%s"%data_check(df_distMat))
print()
print(check_ns(df_distMat))
print()
num_cities = df_distMat.shape[0]
# df_cityDist.set_index('city')
print('Number of cities including depot: ', num_cities)

rows(6) x cols(6) 

      0     1     2     3     4     5
0   NaN  36.0  32.0  54.0  20.0  40.0
1  36.0   NaN  22.0  58.0  54.0  67.0
2  32.0  22.0   NaN  36.0  42.0  71.0

              0          1          2          3          4         5
count   5.00000   5.000000   5.000000   5.000000   5.000000   5.00000
mean   36.40000  47.400000  40.600000  58.000000  42.200000  63.00000
std    12.36123  18.132843  18.487834  20.736441  13.236314  21.05944
min    20.00000  22.000000  22.000000  36.000000  20.000000  40.00000
25%    32.00000  36.000000  32.000000  50.000000  42.000000  45.00000
50%    36.00000  54.000000  36.000000  54.000000  45.000000  67.00000
75%    40.00000  58.000000  42.000000  58.000000  50.000000  71.00000
max    54.00000  67.000000  71.000000  92.000000  54.000000  92.00000

Number of cities including depot:  6


# BIN PACKING PROBLEM
<font color='royalblue'>The Bin Packing Proble (BPP) component entails packing the $n$ deliverable items into the minimum number of bins without exceeding its fixed capacity and has the minimum wastage of space.
 In this case our bins are the subset of available delivery vehicles, into which the items are to be packed.
Note, not all the items that are required to be distributed are the same in size.</font>

# One dimesional
<font color='grass'>
In order to simplify the problem, we initially consider the problem to be one dimensional. Only one dimension of the item is not fixed, for example length, whilst the other two dimensions (width and height) remain constant.
</font>

# Two dimesional
<font color='grass'>
Next we consider the problem to be rwo dimensional. Only one dimension of the item is fixed, for example height, whilst the other two dimensions - width and length -are variable.
</font>

In [10]:
#bin summary
def bin_summary(df_vehicles):
    number_of_bins = df_vehicles.shape[0]
    unused = 0
    partial = 0
    max_fill = 0
    for i in range(number_of_bins):
        av_space = df_vehicles.loc[i,'available_space']
        bin_size = df_vehicles.loc[i, 'size']
        if av_space == bin_size:
            unused = unused + 1
        elif av_space == 0:
            max_fill = max_fill +1
        elif av_space < bin_size and av_space!=0:
            partial = partial+1
            
    print("Number of bins : %d" %(number_of_bins))
    print("Number of partial filled bins : %d" %(partial))
    print("Number of unused bins : %d" %(unused))
    print("Number of max filled bins : %d" %(max_fill))
    return None

In [11]:
#counts the number of unused bins after allocating  items
def unused_bin_2D(df_vehicles):
    number_of_bins = df_vehicles.shape[0]
    count = 0
    for i in range(number_of_bins):
        if df_vehicles.loc[i, 'x_size'] == df_vehicles.loc[i, 'av_x_space'] and df_vehicles.loc[i, 'y_size'] == df_vehicles.loc[i, 'av_x_space']:
            count = count + 1
        else:
            count = count 
            return count

#bin summary
def bin_summary_2D(df_vehicles):
    number_of_bins = df_vehicles.shape[0]
    unused = 0
    partial = 0
    max_fill = 0
    for i in range(number_of_bins):
        av_space_x = df_vehicles.loc[i, 'av_x_space']
        av_space_y = df_vehicles.loc[i, 'av_y_space']
        bin_size_x = df_vehicles.loc[i, 'x_size']
        bin_size_y = df_vehicles.loc[i, 'y_size']
        if av_space_x == bin_size_x and av_space_y == bin_size_y:
            unused = unused + 1
        elif av_space_x == 0 and av_space_y == 0:
            max_fill = max_fill +1
        elif (av_space_x < bin_size_x and av_space_x!=0) or (av_space_y < bin_size_y and av_space_y!=0):
            partial = partial+1
    
    print("Number of bins : %d" %(number_of_bins))
    print("Number of partial filled bins : %d" %(partial))
    print("Number of unused bins : %d" %(unused))
    print("Number of max filled bins : %d" %(max_fill))
            
    return number_of_bins,unused,max_fill, partial

#number of items not accounted for
def unpacked_items(df_items):
    number_of_items = df_items.shape[0]
    not_packed = 0
    for i in range(number_of_items):
        if df_items.loc[i, 'bin'] == nan:
            not_packed = not_packed +1        
    return not_packed

# GREEDY APPROACH
<font color='darkorange'>A technique which makes locally optimal choices with hope of obtaining the global optimum. 
</font>

## Fit algorithms:  

### First-fit algorithm:
<font color='rebeccapurple'>
Algorithm scans the bins for the first bin which has a large enough space to fit the item. If the current bin has adequate space, the items is allocated to the bin, else the next bin is checked. For the next item, we iterate the bins from the FIRST bin (hence first fit algorithm).
</font>

In [12]:
def first_fit(df_subItems, df_items, df_vehicles):
    number_of_subItems = df_subItems.shape[0]
    number_of_bins = df_vehicles.shape[0]
    df_subItems.index = range(0,number_of_subItems)#reindexsubitems    
    for i in range(number_of_subItems):
        item_no = df_subItems.loc[i,'item']
        j = 0
        item_allocated = False
        item_size = df_subItems.loc[i,'size']
        while j<= number_of_bins and item_allocated == False:
            available_bin_space = df_vehicles.loc[j,'available_space']
            if available_bin_space >= item_size: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.loc[j,'available_space'] = df_vehicles.loc[j,'available_space'] - item_size #update avialable space
                bin_num = df_vehicles.loc[j,'bin']
                df_items.loc[item_no,'bin'] = bin_num#set the allocated bin for the item
            elif available_bin_space < item_size: #if NOT adequate space in the curr bin for the item
                #move to the next bin
                j = j+1 
                if j>=number_of_bins: #if none of the bins are large enough to house the item
                    item_allocated = True
                    df_items.loc[item_no,'bin'] = np.nan
    return df_items, df_vehicles

In [13]:
# df_subsetItems = df_items[df_items['bin'].apply(np.isnan)]
# df_subsetItems

In [14]:
# df_items, df_vehicles = first_fit(df_subsetItems, df_items, df_vehicles)
# display(df_items)
# display(df_vehicles)

In [15]:
def first_fit_2D(df_items, df_vehicles):
    number_of_items = df_items.shape[0]
    number_of_bins = df_vehicles.shape[0]
    for i in range(number_of_items):
        j = 0
        item_allocated = False
        item_size_x = df_items.loc[i,'x_size']
        item_size_y = df_items.loc[i,'y_size']
        while j<= number_of_bins and item_allocated == False:
            available_bin_space_x = df_vehicles.loc[j,'av_x_space']
            available_bin_space_y = df_vehicles.loc[j,'av_y_space']
            if available_bin_space_x >= item_size_x and available_bin_space_y >= item_size_y: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.loc[j,'av_x_space'] = df_vehicles.loc[j,'av_x_space'] - item_size_x #update avialable space in x direction
                df_vehicles.loc[j,'av_y_space'] = df_vehicles.loc[j,'av_y_space'] - item_size_y #update avialable space in y direction 
                bin_num = df_vehicles.loc[j,'bin']
                df_items.loc[i,'bin'] = bin_num#set the allocated bin for the item
            elif available_bin_space_x >= item_size_y and available_bin_space_y >= item_size_x: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.loc[j,'av_x_space'] = df_vehicles.loc[j,'av_x_space'] - item_size_y #update avialable space in x direction
                df_vehicles.loc[j,'av_y_space'] = df_vehicles.loc[j,'av_y_space'] - item_size_x #update avialable space in y direction 
                bin_num = df_vehicles.loc[j,'bin']
                df_items.loc[i,'bin'] = bin_num#set the allocated bin for the item
            elif available_bin_space_x < item_size_x or available_bin_space_y < item_size_y: #if NOT adequate space in the curr bin for the item
                #move to the next bin
                j = j+1 
                if j>=number_of_bins: #if none of the bins are large enough to house the item
                    item_allocated = True
                    df_items.loc[i,'bin'] = np.nan
    return df_items, df_vehicles

### Next-fit algorithm
<font color='rebeccapurple'>After allocating first bin large enough to house the item, when looking at the next item to be allocated, find the next suitable bin from the current bin. NOT starting from the very first bin. Check from the next bin in a loop, and stop at the bin before the last allocated (current bin). The search space is from current bin to the previous in a loop.

In [16]:
def next_fit(df_items, df_vehicles):
    number_of_items = df_items.shape[0]
    number_of_bins = df_vehicles.shape[0]
    j = 0
    for i in range(number_of_items):
        item_allocated = False
        item_size = df_items.loc[i,'size']
        while item_allocated == False and j<= number_of_bins:
            available_bin_space = df_vehicles.loc[j,'available_space']
            if available_bin_space >= item_size: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.loc[j,'available_space'] = df_vehicles.loc[j,'available_space'] - item_size #update avialable space
                bin_num = df_vehicles.loc[j,'bin']
                df_items.loc[i,'bin'] = bin_num #set the allocated bin for the item
                j_curr = j
            elif available_bin_space < item_size: #if NOT adequate space in the curr bin for the item
                #move to the next bin
                j = j+1 
                if j>=number_of_bins: #if none of the bins are large enough to house the item
                    j = 0
                    if j_curr == j:
                        item_allocated = True
                        df_items.loc[i,'bin'] = np.nan
                        j = j_curr
    return df_items, df_vehicles

In [17]:
def next_fit_2D(df_items, df_vehicles):
    number_of_items = df_items.shape[0]
    number_of_bins = df_vehicles.shape[0]
    j = 0
    for i in range(number_of_items):
        item_allocated = False
        item_size_x = df_items.loc[i,'x_size']
        item_size_y = df_items.loc[i,'y_size']
        while item_allocated == False and j<= number_of_bins:
            available_bin_space_x = df_vehicles.loc[j,'av_x_space']
            available_bin_space_y = df_vehicles.loc[j,'av_y_space']
            if available_bin_space_x >= item_size_x and available_bin_space_y >= item_size_y: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.loc[j,'av_x_space'] = df_vehicles.loc[j,'av_x_space'] - item_size_x #update avialable space in x direction
                df_vehicles.loc[j,'av_y_space'] = df_vehicles.loc[j,'av_y_space'] - item_size_y #update avialable space in y direction 
                bin_num = df_vehicles.loc[j,'bin']
                df_items.loc[i,'bin'] = bin_num #set the allocated bin for the item
                j_curr = j
            elif available_bin_space_x >= item_size_y and available_bin_space_y >= item_size_x: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.loc[j,'av_x_space'] = df_vehicles.loc[j,'av_x_space'] - item_size_y #update avialable space in x direction
                df_vehicles.loc[j,'av_y_space'] = df_vehicles.loc[j,'av_y_space'] - item_size_x #update avialable space in y direction 
                bin_num = df_vehicles.loc[j,'bin']
                df_items.loc[i,'bin'] = bin_num #set the allocated bin for the item
                j_curr = j
            elif available_bin_space_x < item_size_x or available_bin_space_y < item_size_y: #if NOT adequate space in the curr bin for the item
                #move to the next bin
                j = j+1 
                if j>=number_of_bins: #if none of the bins are large enough to house the item
                    j = 0
                    if j_curr == j:
                        item_allocated = True
                        df_items.loc[i,'bin'] = np.nan
                        j = j_curr
    return df_items, df_vehicles

### Best-fit algorithm
<font color='rebeccapurple'>Allocating item to a in such that there is minimum wastage of space is left in in the bin.
<\font>

In [18]:
def best_fit(df_items, df_vehicles):
    number_of_items = df_items.shape[0]
    number_of_bins = df_vehicles.shape[0]
    for i in range(number_of_items):
        j = 0
        item_allocated = False
        item_size = df_items.loc[i,'size']
        df_vehicles = df_vehicles.sort_values(by='available_space', ascending=1).reset_index(drop=True)#after each iteration, order bins as per min space, so less wastage
        while j<= number_of_bins and item_allocated == False:
            available_bin_space = df_vehicles.loc[j,'available_space']
            if available_bin_space >= item_size: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.loc[j,'available_space'] = df_vehicles.loc[j,'available_space'] - item_size #update avialable space
                bin_num = df_vehicles.loc[j,'bin']
                df_items.loc[i,'bin'] = bin_num#set the allocated bin for the item
            elif available_bin_space < item_size: #if NOT adequate space in the curr bin for the item
                #move to the next bin
                j = j+1 
                if j>=number_of_bins: #if none of the bins are large enough to house the item
                    item_allocated = True
                    df_items.loc[i,'bin'] = np.nan
    return df_items, df_vehicles

In [19]:
def best_fit_2D(df_items, df_vehicles):
    number_of_items = df_items.shape[0]
    number_of_bins = df_vehicles.shape[0]
    for i in range(number_of_items):
        j = 0
        item_allocated = False
        item_size_x = df_items.loc[i,'x_size']
        item_size_y = df_items.loc[i,'y_size']
        df_vehicles = df_vehicles.sort_values(by='x_size', ascending=1).reset_index(drop=True)#after each iteration, order bins as per min space, so less wastage
        while j<= number_of_bins and item_allocated == False:
            available_bin_space_x = df_vehicles.loc[j,'av_x_space']
            available_bin_space_y = df_vehicles.loc[j,'av_y_space']
            if available_bin_space_x >= item_size_x and available_bin_space_y >= item_size_y: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.loc[j,'av_x_space'] = df_vehicles.loc[j,'av_x_space'] - item_size_x #update avialable space in x direction
                df_vehicles.loc[j,'av_y_space'] = df_vehicles.loc[j,'av_y_space'] - item_size_y #update avialable space in y direction 
                bin_num = df_vehicles.loc[j,'bin']
                df_items.loc[i,'bin'] = bin_num#set the allocated bin for the item
            elif available_bin_space_x >= item_size_y and available_bin_space_y >= item_size_x: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.loc[j,'av_x_space'] = df_vehicles.loc[j,'av_x_space'] - item_size_y #update avialable space in x direction
                df_vehicles.loc[j,'av_y_space'] = df_vehicles.loc[j,'av_y_space'] - item_size_x #update avialable space in y direction 
                bin_num = df_vehicles.loc[j,'bin']
                df_items.loc[i,'bin'] = bin_num#set the allocated bin for the item
            elif available_bin_space_x < item_size_x or available_bin_space_y < item_size_y: #if NOT adequate space in the curr bin for the item
                #move to the next bin
                j = j+1 
                if j>=number_of_bins: #if none of the bins are large enough to house the item
                    item_allocated = True
                    df_items.loc[i,'bin'] = np.nan
    return df_items, df_vehicles

### Worst-fit algorithm
<font color='orange'>Allocating item to a bin such that there is maximum wastage of space is left in in the bin.
<\font>

In [20]:
def worst_fit(df_items, df_vehicles):
    number_of_items = df_items.shape[0]
    number_of_bins = df_vehicles.shape[0]
    for i in range(number_of_items):
        j = 0
        item_allocated = False
        item_size = df_items.loc[i,'size']
        df_vehicles = df_vehicles.sort_values(by='available_space', ascending=0).reset_index(drop=True)#after each iteration, order bins as per min space, so less wastage
        while j<= number_of_bins and item_allocated == False:
            available_bin_space = df_vehicles.loc[j,'available_space']
            if available_bin_space >= item_size: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.loc[j,'available_space'] = df_vehicles.loc[j,'available_space'] - item_size #update avialable space
                bin_num = df_vehicles.loc[j,'bin']
                df_items.loc[i,'bin'] = bin_num#set the allocated bin for the item
            elif available_bin_space < item_size: #if NOT adequate space in the curr bin for the item
                #move to the next bin
                j = j+1 
                if j>=number_of_bins: #if none of the bins are large enough to house the item
                    item_allocated = True
                    df_items.loc[i,'bin'] = np.nan
    return df_items, df_vehicles

In [21]:
def worst_fit_2D(df_items, df_vehicles):
    number_of_items = df_items.shape[0]
    number_of_bins = df_vehicles.shape[0]
    for i in range(number_of_items):
        j = 0
        item_allocated = False
        item_size_x = df_items.loc[i,'x_size']
        item_size_y = df_items.loc[i,'y_size']
        df_vehicles = df_vehicles.sort_values(by='x_size', ascending=0).reset_index(drop=True)#after each iteration, order bins as per min space, so less wastage
        while j<= number_of_bins and item_allocated == False:
            available_bin_space_x = df_vehicles.loc[j,'av_x_space']
            available_bin_space_y = df_vehicles.loc[j,'av_y_space']
            if available_bin_space_x >= item_size_x and available_bin_space_y >= item_size_y: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.loc[j,'av_x_space'] = df_vehicles.loc[j,'av_x_space'] - item_size_x #update avialable space in x direction
                df_vehicles.loc[j,'av_y_space'] = df_vehicles.loc[j,'av_y_space'] - item_size_y #update avialable space in y direction 
                bin_num = df_vehicles.loc[j,'bin']
                df_items.loc[i,'bin'] = bin_num#set the allocated bin for the item
            elif available_bin_space_x >= item_size_y and available_bin_space_y >= item_size_x: #if adequate space in the bin for the item
                item_allocated = True
                df_vehicles.loc[j,'av_x_space'] = df_vehicles.loc[j,'av_x_space'] - item_size_y #update avialable space in x direction
                df_vehicles.loc[j,'av_y_space'] = df_vehicles.loc[j,'av_y_space'] - item_size_x #update avialable space in y direction 
                bin_num = df_vehicles.loc[j,'bin']
                df_items.loc[i,'bin'] = bin_num#set the allocated bin for the item
            elif available_bin_space_x < item_size_x or available_bin_space_y < item_size_y: #if NOT adequate space in the curr bin for the item
                #move to the next bin
                j = j+1 
                if j>=number_of_bins: #if none of the bins are large enough to house the item
                    item_allocated = True
                    df_items.loc[i,'bin'] = np.nan
    return df_items, df_vehicles

### Fit Algorithms considering priority values
In order to pack items with highest priority first and to meet delivery deadlines, we follow the following implementation:
1. Create a subset of items which have a delivery deadline 
2. Create a copy of the bin list for each delivery dispatch 
3. Allocate items of that particular deadline to the the bins
4. If not all items are allocated to bins, then create a dummy bin to which it items are allocaed to (size = largest size from currently available bins) 
5. Run the algorithm on later deadline delivery items 

The algorithm returns the item dataframe with bins it is allocated and the deliery dispatch round. There are bin dataframes returned, specific to each dispatch round. 

In [22]:
# # INPUT: Datframes items and bins. number of dispatch rounds, in this particular case we have two
# # OUTPUT: Dataframes items and bins corresponding to each delivery round

# def priority_binPacking_firstFit1D(df_items, df_bins):
#     df_items_group1 = df_items[df_items.delivery_time == 1]#delivery time 1
#     df_items_group2 = df_items[df_items.delivery_time == 2]#delivery time 2
#     df_binSet = df_bins.copy()#make a copy of bin set
#     df_bins_dispatch1 = df_binSet.copy()#bin set for dispatch one
#     df_bins_dispatch2 = df_binSet.copy()#bin set for dispatch 2
#     num_bins = df_bins.shape[0]
    
#     #create a new dataframe with the largest bin. A bin of this size is added to the dispatch batch if an extra bin is required
#     num_bin_attributes = df_bins.shape[1]
#     extra_bin = pd.DataFrame(np.zeros((1,df_bins.shape[1])), columns=df_bins.columns)
#     row_largest_bin = df_bins.idxmax()
#     extra_bin.loc[0,:] = df_bins.loc[row_largest_bin,:] # let the dummy bin be the largest bin that wehave available from the set
    
#     #assign dispatch1 items to bins
#     df_items_group1, df_bins_dispatch1 = first_fit(df_items_group1, df_bins_dispatch1)
#     #check for any items that have not been assigned
#     nan_index = df_items_group1['bin'].index[df_items_group1['bin'].apply(np.isnan)]
#     nan_index_list = nan_index.values.tolist()#creates a list of all the items which have not been assigned bins
#     num_unallocated_items = len(nan_index_list)
    
#     while num_unallocated_items != 0:#while there are unallocated items
#         #add bin 
#         num_bins = num_bins + 1
#         df_bins_dispatch1 = df_bins_dispatch1.append(extra_bin)
#         df_bins_dispatch1.index = range(0,num_bins)#reindex#change bin number and index according to the number of adds of extra bins
#         df_bins_dispatch1['bin'] = df_bins_dispatch1.index
#         #run algorithm from the START(ie, even the items which were added)
#         df_items_group1, df_bins_dispatch1 = first_fit(df_items_group1, df_bins_dispatch1)
#         #calculate number of unallocated items
#         nan_index = df_items_group1['bin'].index[df_items_group1['bin'].apply(np.isnan)]
#         nan_index_list = nan_index.values.tolist()#creates a list of all the items which have not been assigned bins
#         num_unallocated_items = len(nan_index_list)
        
    
    
    
    
#     return df_items, df_bin_dispatch1, df_bin_dispatch2

In [23]:
# INPUT: Datframes items and bins. number of dispatch rounds, in this particular case we have two
# OUTPUT: Dataframes items and bins corresponding to each delivery round

def priority_binPacking_firstFit1D(df_items, df_bins):
    df_items_group1 = df_items[df_items.delivery_time == 1]#delivery time 1
    
    df_binSet = df_bins.copy()#make a copy of bin set
    df_bins_dispatch1 = df_binSet.copy()#bin set for dispatch one
    df_bins_dispatch2 = df_binSet.copy()#bin set for dispatch 2
    num_bins = df_bins.shape[0]
           
    #create a new dataframe with the largest bin. A bin of this size is added to the dispatch batch if an extra bin is required
    num_bin_attributes = df_bins.shape[1]
    extra_bin = pd.DataFrame(np.zeros((1,df_bins.shape[1])), columns=df_bins.columns)
    largest_bin = df_bins.idxmax()
    extra_bin.loc[0,:] = largest_bin # let the dummy bin be the largest bin that wehave available from the set
    
    
    df_items_group1_unallocated = df_items_group1
    count = 0#counts iteration of adding bins
    while df_items_group1_unallocated.shape[0] !=0:# while there are items that have to be allocated bins to meet dealine
        if count ==0:#round 0, no extra bins added
            df_items, df_bins_dispatch1 = first_fit(df_items_group1_unallocated, df_items, df_bins_dispatch1) # algorithm to add aallocate items to bin
            count = count+1
            df_items_group1_unallocated = df_items[(df_items.delivery_time == 1) & (df_items.bin.apply(np.isnan))] #groups all items which are part of delivery one and are unallocated, no bin allocation
        elif count >0:
            #add extra bin
            df_bins_dispatch1 = df_bins_dispatch1.append(extra_bin)
            df_bins_dispatch1.index = range(0,num_bins+count)#reindex#change bin number and index according to the number of adds of extra bins
            df_bins_dispatch1['bin'] = df_bins_dispatch1.index
            #run algorithm on remaining items
            df_items, df_bins_dispatch1 = first_fit(df_items_group1_unallocated, df_items, df_bins_dispatch1) # algorithm to add aallocate items to bin
            #account for round and unallocated items
            count = count+1
            df_items_group1_unallocated = df_items[(df_items.delivery_time == 1) & (df_items.bin.apply(np.isnan))] #groups all items which are part of delivery one and are unallocated, no bin allocation
            
    #after all group one items have been added to bins, next we run the algorithm for items in group 2
    df_items_group2 = df_items[(df_items.delivery_time == 2) & (df_items.delivery_dispatch.apply(np.isnan))]#delivery time 2
    df_items_group2_unallocated = df_items_group2
    df_items, df_bins_dispatch1 = first_fit(df_items_group2_unallocated, df_items, df_bins_dispatch1)
    df_items_group2_unallocated = df_items[(df_items.delivery_time == 2) & (df_items.bin.apply(np.isnan))]
    #set all the items dispatched in round 1
    df_items.loc[df_items['bin'].apply(np.isfinite), 'delivery_dispatch'] = 1
    df_bins_dispatch1.loc[:,'delivery_time_dispatch'] = 1
    
    
    #run algorithm for all unallocated items in group 2
    count = 0
    while df_items_group2_unallocated.shape[0] !=0:# while there are items that have to be allocated bins to meet dealine
        if count ==0:#round 0, no extra bins added
            df_items, df_bins_dispatch2 = first_fit(df_items_group2_unallocated, df_items, df_bins_dispatch2) # algorithm to add aallocate items to bin
            count = count+1
            df_items_group2_unallocated = df_items[(df_items.delivery_time == 2) & (df_items.bin.apply(np.isnan))] #groups all items which are part of delivery one and are unallocated, no bin allocation
        elif count >0:
            #add extra bin
            df_bins_dispatch2 = df_bins_dispatch2.append(extra_bin)
            df_bins_dispatch2.index = range(0,num_bins+count)#reindex#change bin number and index according to the number of adds of extra bins
            df_bins_dispatch2['bin'] = df_bins_dispatch2.index
            #run algorithm on remaining items
            df_items, df_bins_dispatch2 = first_fit(df_items_group2_unallocated, df_items, df_bins_dispatch2) # algorithm to add aallocate items to bin
            #account for round and unallocated items
            count = count+1
            df_items_group2_unallocated = df_items[(df_items.delivery_time == 2) & (df_items.bin.apply(np.isnan))] #groups all items which are part of delivery one and are unallocated, no bin allocation
    #set all the items dispatched in round 2
    df_items.loc[df_items['delivery_dispatch'].apply(np.isnan), 'delivery_dispatch'] = 2
    df_bins_dispatch2.loc[:,'delivery_time_dispatch'] = 2
 
    
    return df_items, df_bins_dispatch1, df_bins_dispatch2

In [24]:
df_items1, df_bin_dispatch1, df_bin_dispatch2 = priority_binPacking_firstFit1D(df_items, df_vehicles)
display(df_items1)


KeyboardInterrupt: 

In [ ]:
display(df_bin_dispatch1)
display(df_bin_dispatch2)

# The Traveling Salesman Problem (TSP)
#### The TSP depicts a salesman who has to visit $n$ cities, returning to it's start  (home)  city, whilst not visiting any of the nodes more than once.
# Greedy Approach
<font color='darkorange'>A technique which makes locally optimal choices with hope of obtaining the global optimum. 
</font>
 
Stores data in an adjacency matrix.

DistPsuedo-code for nearest neighbour algorithm:
1. From start node (V0), find the nearest neighbour (Vn). In the zeroth row from col 1:end find the smallest distance.
2. Put the next visited neighbour in the terminal set -> T. Add the new vertes  to the the possible set -> P. (Note: this must be assigned to Vn)
3. Do this while D(V0) = 0

<font color='blue'>Ammended to take in array of states to visit and return the order of route and the distance
</font>

In [ ]:
# Input: Array of states that need to be visited by the bin
    #arr_states does not include the depots
# OUTPUT: Array with order of states to vist and distance

def nearest_neighbour_arr(arr_states, df_distMatrix):
        num_states = len(arr_states)
        total_states = df_distMatrix.shape[0]
        
        #creates an array with depot at the begining
        x0 = np.array([0])
        arr_all_states = np.concatenate((x0,arr_states,x0), axis = 0)
        
        #create a possible dataframe of length of the num_states
        possible = pd.DataFrame(np.zeros((num_states+2,1)))# Accounts for the vertices that have been visited
        possible.index.name = 'possible'
        possible = possible.reindex(arr_all_states)
        possible.iloc[:,:] = True
        possible.iloc[0,0]= False
        
        #distance calculator
        distances = pd.DataFrame(np.zeros((num_states+2,1)))# The distance from Vertex i to the next vertex
        distances.index.name = 'distances'
        distances = distances.reindex(arr_all_states)
        distances.iloc[:,:] = 0
        
        #terminal set 
        terminal_set = pd.DataFrame(np.zeros((num_states,1)))# The set/ordering of vertices that are encountered on the route
        terminal_set.loc[0,0] = 0
        terminal_set.loc[num_states,0] = 0
        
        #create a distance matrix with only the columns that can be visited
        df_distMatrix.fillna(inf,inplace = True)
        df_curr_distMatrix = pd.DataFrame(np.zeros((total_states, total_states)))
        df_curr_distMatrix.iloc[:,:] = inf
        
        for i in range(num_states):
            state = arr_states[i]
            df_curr_distMatrix.iloc[:,state] = df_distMatrix.iloc[:, state]
        #add the depot 
        df_curr_distMatrix.iloc[:,0] = df_distMatrix.iloc[:,0]#begining depot
        #df_curr_distMatrix.iloc[:,total_states+1] = df_distMatrix.iloc[:,0]#end depot
        
        #find the nearest neighbour using this updated distance matrix
        count = 0
        new_curr = 0 #depot is the first vertex
        #Note: iloc indexes from 0 to n-1 
        while possible.iloc[num_states+1,0]== True and count<num_states+2: 
            curr = new_curr
            if count < num_states:
                nearest_vertex = df_curr_distMatrix.loc[curr,1:].idxmin()#gives the minimum values index ie column
                nearest_vertex = int(nearest_vertex)
                if possible.loc[nearest_vertex,0]==True: #if the next vertex is not used
                    dist = df_curr_distMatrix.iloc[curr,nearest_vertex]
                    distances.loc[curr,0] = dist
                    possible.loc[nearest_vertex,0]=False
                    terminal_set.iloc[count,0] = nearest_vertex
                    df_curr_distMatrix.iloc[:,nearest_vertex] = inf
                    new_curr = nearest_vertex
                    count = count+1
                elif possible.loc[nearest_vertex,0]==False:#if the next vertex is used
                    df_curr_distMatrix.iloc[curr,nearest_vertex]=inf
            elif count == num_states:
                nearest_vertex = 0
                dist = df_curr_distMatrix.loc[curr,0]
                distances.iloc[num_states,0] = dist
                possible.iloc[num_states+1,0] = False
                terminal_set.loc[count,0] = 0
                count = count +1
        arr_terminal_route_temp  = terminal_set.loc[:,0].values
        x0 = np.array([0])
        arr_terminal_route = np.concatenate((x0, arr_terminal_route_temp), axis = 0)
        distance = distances.iloc[0:num_states+1,0].sum() 
        return arr_terminal_route, distance


#  METAHEURASTIC: SIMULATED ANNEALING
<font color='darkorange'>Simulated annealing (SA) is a probabilistic technique for approximating the global optimum of a given function. Specifically, it is a metaheuristic to approximate global optimization in a large search space. It is often used when the search space is discrete (e.g., all tours that visit a given set of cities).
</font>

# FULL IMPLEMENTATION 
### Components
1. Group by value in a column
2. Convert a dataframe column to an array
3. Finding corresponding value in a column and assignning the order number

In [ ]:
# ## Displays rows of a dataframe where the values in a particular column are as specifies
# # INPUT: DataFrame, column name, value grouping by 
# # OUTPUT: Subet of the dataframe where the column specified has the value by which it's being grouped
# def groupby_value(df, col_name, value):
#     df_grouped = df[df.col_name == value]
#     return df_grouped

In [ ]:
##PUT ITEMS IN ARRAY WHICH ARE GROUPED BY SOME VALUE IN A COLUMN
#INPUT: a grouped by bin item dataframe
        #original bin dataframe
        #bin number by which items are grouped 
#OUTPUT: return an array of delivery locations of that bin
        #to fill the route of that bin in the bin_dataframe
def dfCol_arr(df_grouped_items):
    arr_delivery_loc = df_grouped_items.loc[:,'delivery_loc'].values
    return arr_delivery_loc


In [ ]:
## Using the subset of the grouped values dataframe, find the correspoding location in the array and fill in the order/delivery number on the delivery route
# INPUT: Grouped dataframe, original dataframe, array- route, key_col_name, col_update_name, search_col
        #df = df_items
        #df_group = subset of items in the same bin
        #arr_route = route produced by greedy algorithm/ SA metaheurastic including the depot at the start and end.
        #key_col = delivery_loc
        #search_col = item
        #update_col = delivery_number
# OUTPUT: Updated original dataframe
    #df = df_items with delivery number allocated to each item

def assign_order(df, df_group, arr_route, key_col, search_col, update_col): #key_col = delivery_loc,#search_col = items #update_col = delivery_number
    arr_len = len(arr_route)
    for i in range(1, arr_len-1):#exclude depots
        state = arr_route[i]
        item_row = df_group.loc[df_group['%s'%key_col] == state].index[0]#row number of location of the state
        item = df_group.loc[item_row, '%s'%search_col]
        df.loc[item, '%s'%update_col] = i #assign order
    return df

Once all items ahve been assigned to a bin, group items by bin. Get all the locations in array format. Using the  Greeedy algorithm or the SA metaheaurastic, generate a route and the cost of the route. 
Set the corresponding states, delivery number in the items dataframe. 
Set the route cost for the particular bin in the bin dataframe. 
<font color='red'>SA to be ammended!
</font>

In [ ]:
# INPUT: item (where all items have been assigned bins) and bin dataframe
# OUTPUT: updated item and bin dataframes, with delivery number and cost of the route

def del_order_cost(df_items, df_bins, df_dist_matrix):
    num_bins = df_bins.shape[0]
    for i in range(num_bins):
        bin_no = i
        df_item_subset = df_items[df_items.bin == bin_no] #subset of items in the bin of value bin_no
        arr_del_loc = dfCol_arr(df_item_subset)# array of all delivery locations
        #ROUTE AND ROUTE DISTANCE
        #nearest neighbour
        arr_route_set, route_dist = nearest_neighbour_arr(arr_del_loc, df_dist_matrix)
        #Simulated annealing
#         arr_route_set, route_dist = simulated_anealing(adj_mat, num_cities, distances, possible,terminal_set)
        #route distance and cost
        df_bins.loc[i, 'route_dist'] = route_dist #setting the route 
        bin_penalty_factor = df_bins.loc[i, 'bin_penalty'] #the penalty associated with the bin
        df_bins.loc[i, 'route_cost'] = route_dist*bin_penalty_factor  #route cost = route dist X bin penalty
        #delivery number on route
        df_items = assign_order(df_items, df_item_subset, arr_route_set, 'delivery_loc', 'item', 'delivery_number')
    return df_items, df_bins